In [1]:
import pandas as pd
import pymysql

# your username and password go here!
creds = {'user': 'brian', 
         'pswd': 'farley'} 


connection = pymysql.connect(host="atriskwiki.cbprs6vpqcbz.us-west-1.rds.amazonaws.com",
                            user=creds['user'],
                            password=creds['pswd'],
                            port=3306,
                            db="atriskwiki",
                            charset="utf8")

cur = connection.cursor()

## Included features:

core.page_len  
aggrevivisionsandviews.revisionslastyear  
aggrevivisionsandviews.pageviews  
date_time_features.days_since_touch  
date_time_features.noDates  
date_time_features.medianWithinMonth  
date_time_features.medianWithinWeek  
date_time_features.medianInFuture  
date_time_features.mostRecentWithinMonth  
date_time_features.mostRecentWithinWeek  
date_time_features.mostRecentInFuture  
exlink_features.exLink_count  
exlink_features.frac_database  
exlink_features.frac_entertainment  
exlink_features.frac_library  
exlink_features.frac_newspaper  
exlink_features.frac_popular  
exlink_features.frac_scholarly  
exlink_features.frac_social  
exlink_features.frac_wiki  
pagelinks.pl_count  
pagelinks.ratio / 1000  
categorylinks:   
    everything except N_births categories, categories starting with _, other splitting artifacts  
all wordfeatures  

In [2]:
tempcore = """CREATE TEMPORARY TABLE tset
              SELECT page_id, page_is_ep, page_len
              FROM core
              WHERE page_in_train = 1 AND
              page_batch < 99"""
cur.execute(tempcore)



62491

In [3]:
gettempcore = """SELECT page_id, page_is_ep, page_len
                 FROM tset"""
cur.execute(gettempcore)
masterData = pd.DataFrame(list(cur.fetchall()), columns=['page_id','page_is_ep','page_len'])
masterData

,page_id,page_is_ep,page_len
0,25,1,146470
1,290,1,19399
2,307,1,171988
3,308,1,109151
4,316,0,85839
5,572,0,13686
6,594,1,126233
7,620,1,66673
8,621,0,129617
9,628,1,48260


In [4]:
getViews = """SELECT page_id, revisionslastyear, pageviews FROM
              tset INNER JOIN aggrevisionsandviews USING (page_id)"""
cur.execute(getViews)
masterData = pd.merge(masterData,pd.DataFrame(list(cur.fetchall()),columns=['page_id','revisionslastyear','pageviews']),on='page_id',how='outer')
masterData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62491 entries, 0 to 62490
Data columns (total 5 columns):
page_id              62491 non-null int64
page_is_ep           62491 non-null int64
page_len             62491 non-null int64
revisionslastyear    57230 non-null float64
pageviews            60404 non-null float64
dtypes: float64(2), int64(3)
memory usage: 2.9 MB


In [5]:
dateCols = ['page_id','days_since_touch','noDates','medianWithinWeek','medianWithinMonth','medianInFuture',
            'mostRecentWithinMonth','mostRecentWithinWeek','mostRecentInFuture']
getDates = """SELECT {}
              FROM tset INNER JOIN date_time_features USING (page_id)""".format(",".join(dateCols))
cur.execute(getDates)
masterData = pd.merge(masterData,pd.DataFrame(list(cur.fetchall()),columns=dateCols),on='page_id',how='outer')
masterData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62491 entries, 0 to 62490
Data columns (total 13 columns):
page_id                  62491 non-null int64
page_is_ep               62491 non-null int64
page_len                 62491 non-null int64
revisionslastyear        57230 non-null float64
pageviews                60404 non-null float64
days_since_touch         62487 non-null float64
noDates                  62487 non-null float64
medianWithinWeek         62487 non-null float64
medianWithinMonth        62487 non-null float64
medianInFuture           62487 non-null float64
mostRecentWithinMonth    62487 non-null float64
mostRecentWithinWeek     62487 non-null float64
mostRecentInFuture       62487 non-null float64
dtypes: float64(10), int64(3)
memory usage: 6.7 MB


In [6]:
exLinkCols = ['page_id','exLink_count','frac_database','frac_entertainment','frac_library','frac_newspaper',
              'frac_popular','frac_scholarly','frac_social','frac_wiki']
getExlinks = """SELECT {}
              FROM tset INNER JOIN exlink_features USING (page_id)""".format(",".join(exLinkCols))
cur.execute(getExlinks)
masterData = pd.merge(masterData,pd.DataFrame(list(cur.fetchall()),columns=exLinkCols),on='page_id',how='outer')
masterData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62491 entries, 0 to 62490
Data columns (total 22 columns):
page_id                  62491 non-null int64
page_is_ep               62491 non-null int64
page_len                 62491 non-null int64
revisionslastyear        57230 non-null float64
pageviews                60404 non-null float64
days_since_touch         62487 non-null float64
noDates                  62487 non-null float64
medianWithinWeek         62487 non-null float64
medianWithinMonth        62487 non-null float64
medianInFuture           62487 non-null float64
mostRecentWithinMonth    62487 non-null float64
mostRecentWithinWeek     62487 non-null float64
mostRecentInFuture       62487 non-null float64
exLink_count             60275 non-null float64
frac_database            60275 non-null float64
frac_entertainment       60275 non-null float64
frac_library             60275 non-null float64
frac_newspaper           60275 non-null float64
frac_popular             60275 no

In [7]:
#my date parsing code forgot to report anything about pages without any external links; also, external links are
#counted by wikipedia ONLY if they're in the main text (I think)
masterData[exLinkCols] = masterData[exLinkCols].fillna(0)
masterData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62491 entries, 0 to 62490
Data columns (total 22 columns):
page_id                  62491 non-null int64
page_is_ep               62491 non-null int64
page_len                 62491 non-null int64
revisionslastyear        57230 non-null float64
pageviews                60404 non-null float64
days_since_touch         62487 non-null float64
noDates                  62487 non-null float64
medianWithinWeek         62487 non-null float64
medianWithinMonth        62487 non-null float64
medianInFuture           62487 non-null float64
mostRecentWithinMonth    62487 non-null float64
mostRecentWithinWeek     62487 non-null float64
mostRecentInFuture       62487 non-null float64
exLink_count             62491 non-null float64
frac_database            62491 non-null float64
frac_entertainment       62491 non-null float64
frac_library             62491 non-null float64
frac_newspaper           62491 non-null float64
frac_popular             62491 no

In [8]:
pLinkCols = ['page_id','pl_count','ratio']
getPlinks = """SELECT {}
              FROM tset INNER JOIN pagelinks USING (page_id)""".format(",".join(pLinkCols))
cur.execute(getPlinks)
masterData = pd.merge(masterData,pd.DataFrame(list(cur.fetchall()),columns=pLinkCols),on='page_id',how='outer')
masterData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62491 entries, 0 to 62490
Data columns (total 24 columns):
page_id                  62491 non-null int64
page_is_ep               62491 non-null int64
page_len                 62491 non-null int64
revisionslastyear        57230 non-null float64
pageviews                60404 non-null float64
days_since_touch         62487 non-null float64
noDates                  62487 non-null float64
medianWithinWeek         62487 non-null float64
medianWithinMonth        62487 non-null float64
medianInFuture           62487 non-null float64
mostRecentWithinMonth    62487 non-null float64
mostRecentWithinWeek     62487 non-null float64
mostRecentInFuture       62487 non-null float64
exLink_count             62491 non-null float64
frac_database            62491 non-null float64
frac_entertainment       62491 non-null float64
frac_library             62491 non-null float64
frac_newspaper           62491 non-null float64
frac_popular             62491 no

In [9]:
import re
cur.execute("SHOW COLUMNS FROM categorylinks")
cols = []
for colname in list(cur.fetchall()):
    col = colname[0]
    okay = True
    if len(col) < 5: #short category names
        okay = False
    if re.match(r"_",col): #leading underscores
        okay = False
    if re.match(r"\d{4}",col): #NNNN_births
        okay = False
    if re.match(r"s_",col): #s_ split artifacts (from preceding apostrophes?)
        okay = False
    if re.search(r"_$",col): #trailing underscore split artifacts
        okay = False
    if okay:
        cols.append(col)
catCols = cols

In [10]:
getClinks = """SELECT {}
              FROM tset INNER JOIN categorylinks USING (page_id)""".format(",".join(catCols))
cur.execute(getClinks)
masterData = pd.merge(masterData,pd.DataFrame(list(cur.fetchall()),columns=catCols),on='page_id',how='outer')
masterData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62491 entries, 0 to 62490
Data columns (total 91 columns):
page_id                                               62491 non-null int64
page_is_ep                                            62491 non-null int64
page_len                                              62491 non-null int64
revisionslastyear                                     57230 non-null float64
pageviews                                             60404 non-null float64
days_since_touch                                      62487 non-null float64
noDates                                               62487 non-null float64
medianWithinWeek                                      62487 non-null float64
medianWithinMonth                                     62487 non-null float64
medianInFuture                                        62487 non-null float64
mostRecentWithinMonth                                 62487 non-null float64
mostRecentWithinWeek                              

In [11]:
masterData['revisionslastyear'] = masterData['revisionslastyear'].fillna(0)

In [12]:
cur.execute("SHOW COLUMNS FROM categorylinks_split")
clsCols = []
for colentry in list(cur.fetchall()):
    clsCols.append(colentry[0])

getCLS = """SELECT {}
            FROM tset INNER JOIN categorylinks_split USING (page_id)""".format(",".join(clsCols))
cur.execute(getCLS)
masterData = pd.merge(masterData,pd.DataFrame(list(cur.fetchall()),columns=clsCols),on='page_id',how='outer')

In [13]:
sparse_master = masterData.dropna(axis='index').to_sparse(fill_value=0)

In [14]:
import pickle
finalWordFeaturesPath = "../WikiTextScraper/FinalWordFeatures.p"
finalWordFeatures = pickle.load(open(finalWordFeaturesPath,"rb"))


In [15]:
finalWordFeatures['page_id'] = finalWordFeatures.index.astype(int)
finalWordFeatures.reset_index(inplace=True)

In [16]:
finalWordFeatures.drop('article_id',axis=1)

,de,popul,town,divis,road,section,draw,district,race,alleg,...,afb,chancel,cdp,voivodeship,northbound,fourlan,rq,tq,gmina,page_id
0,0.000000,0.000782,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000130,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,25.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,290.0
2,0.000000,0.000000,0.000200,0.000067,0.000133,0.000000,0.000266,0.000200,0.000133,0.000133,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,307.0
3,0.000370,0.000000,0.000000,0.000370,0.000000,0.000000,0.000185,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,308.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,316.0
5,0.000763,0.001527,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,572.0
6,0.000085,0.000000,0.000340,0.000000,0.000170,0.000000,0.000085,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,594.0
7,0.000000,0.000000,0.000000,0.011080,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,615.0
8,0.000328,0.000000,0.000000,0.000000,0.000000,0.000000,0.000164,0.000000,0.000164,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,620.0
9,0.000086,0.000599,0.000000,0.000000,0.000000,0.000000,0.000342,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,621.0


In [17]:
wordFeatures = finalWordFeatures.to_dense()
wordFeatures['page_id'] = wordFeatures['page_id'].astype(int)

In [18]:
masterData = pd.merge(masterData, wordFeatures, on='page_id')

In [19]:
masterData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62487 entries, 0 to 62486
Columns: 1523 entries, page_id to gmina
dtypes: float64(1520), int64(3)
memory usage: 726.5 MB


In [21]:
cur.execute("SHOW COLUMNS FROM PCA_word_features")
PCACols = []
for colentry in list(cur.fetchall()):
    PCACols.append(colentry[0])

getPCA = """SELECT {}
            FROM tset INNER JOIN PCA_word_features USING (page_id)""".format(",".join(PCACols))
cur.execute(getPCA)
PCAfeatures = pd.DataFrame(list(cur.fetchall()),columns=PCACols)
PCAfeatures.fillna(0,inplace=True)
masterData = pd.merge(masterData,PCAfeatures,on='page_id',how='outer')

In [24]:
masterData['pl_count'] = masterData['pl_count'].divide(masterData['page_len'])

In [25]:
masterData['exLink_count'] = masterData['exLink_count'].divide(masterData['page_len'])

In [26]:
masterData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62487 entries, 0 to 62486
Columns: 1573 entries, page_id to pca_islam_in_pakistan
dtypes: float64(1570), int64(3)
memory usage: 750.4 MB


In [27]:
sparseMaster = masterData.to_sparse(fill_value=0)

In [28]:
pickle.dump(sparseMaster,open("072716_COMBINED_DATA_TRAIN.p","wb"))

In [29]:
sparseMaster.info()

<class 'pandas.sparse.frame.SparseDataFrame'>
Int64Index: 62487 entries, 0 to 62486
Columns: 1573 entries, page_id to pca_islam_in_pakistan
dtypes: float64(1573)
memory usage: 21.2 MB
